DL9 (gr.1) Learning binarized neural networks using the approach given in [M.Courbariaux]. Comparison of the Adam and RMSProp versions of the SGD algorithm proposed therein. The comparison should be performed on the example of learning LSTM model discussed in [XuanLiu]. Project for 3 students.

In [219]:
import keras
import numpy as np
from tensorflow.keras.layers import LSTM
import tensorflow.compat.v2 as tf
import BinaryLSTM
import importlib
importlib.reload(BinaryLSTM)
import larq

In [220]:
inputs = tf.random.normal([32, 10, 8])
lstm_b = tf.keras.layers.LSTM(4, kernel_constraint='weight_clip', activation='ste_sign', kernel_regularizer='ste_sign', recurrent_activation='ste_sign', return_sequences=True)
lstm_k = tf.keras.layers.LSTM(4, kernel_constraint='weight_clip')
lstm_a = tf.keras.layers.LSTM(4, activation='ste_sign')
lstm = tf.keras.layers.LSTM(4)
output = lstm_b(inputs)
print(output.shape)

(32, 10, 4)


In [221]:
inputs = keras.Input(shape=(10,8))
b = BinaryLSTM.BinaryLSTM(16)
outputs = b(inputs)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
model.compile(loss='MSE', optimizer='adam')
outputTensor = model.output
listOfVariableTensors = model.trainable_weights
gradients = tf.GradientTape(outputTensor, listOfVariableTensors)

In [222]:
inp = tf.random.normal([32, 64])
z0_t, z1_t, z2_t, z3_t = tf.split(inp, 4, axis=1)
out = tf.concat([z0_t, z1_t, z2_t, z3_t], axis=1)
np.all(tf.equal(inp, out))
print(tf.math.exp(tf.zeros((64,))).shape)

(64,)


In [223]:
inputs = tf.random.normal([32, 10, 8])
out1 = BinaryLSTM.BinaryLSTM(16)(inputs)
out1

<tf.Tensor: shape=(32, 16), dtype=float32, numpy=
array([[ 1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,
         1.,  1.,  1.],
       [ 1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,
        -1.,  1., -1.],
       [ 1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1., -1., -1.],
       [ 1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,
        -1.,  1., -1.],
       [-1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1., -1.,
         1., -1.,  1.],
       [ 1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,
         1.,  1.,  1.],
       [ 1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,
        -1.,  1., -1.],
       [-1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,
        -1.,  1.,  1.],
       [-1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,
         1., -1.,  1.],
       [-1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,
    

In [224]:
inputs = tf.random.normal([32, 10, 8])
with tf.GradientTape() as g:
  g.watch(inputs)
  y = BinaryLSTM.BinaryLSTM(16)(inputs)
dy_dx = g.gradient(y, inputs)
print(dy_dx)

tf.Tensor(
[[[ 0.45142138 -0.2458537  -0.28020203 ...  0.35869744 -0.5384949
   -0.44749865]
  [ 0.0495259  -0.018594    0.08185412 ... -0.01225848  0.03259047
   -0.02242724]
  [ 0.0037807  -0.07844369 -0.12362617 ... -0.1227022  -0.2848277
   -0.1709145 ]
  ...
  [ 0.42548206  0.08501267  0.37816012 ... -0.2968651  -0.21102858
    0.17569351]
  [ 0.28252596  0.08128074  0.3152091  ... -0.10929003  0.5632531
    0.6449992 ]
  [ 0.17548981  0.4651398   0.46033582 ... -0.10123844 -0.46574366
    0.2589924 ]]

 [[-0.27280724  0.14000072 -0.17232655 ...  0.27411962  0.04176622
    0.13223146]
  [ 0.11203885  0.09496566 -0.2633737  ...  0.14942166 -0.04647173
    0.12308045]
  [-0.06782056 -0.2183868   0.09999195 ... -0.03300903  0.14818606
    0.06107198]
  ...
  [ 1.1529809   1.8071938   1.083144   ...  0.22282954  0.7372181
    0.44980335]
  [-0.40777838  1.5242898  -0.04192023 ... -1.216842    0.8565004
    1.380747  ]
  [-0.06490961  0.8748127  -0.45015353 ...  0.44675556  0.6751566
 

# This should deal with clipping weights

In [225]:
tf.random.set_seed(0)
inputs = tf.random.normal([4, 10, 8])
b = BinaryLSTM.BinaryLSTM(16, recurrent_constraint='weight_clip', kernel_constraint='weight_clip')
print(b(inputs))

tf.Tensor(
[[ 1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [-1.  1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.]
 [-1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]], shape=(4, 16), dtype=float32)


# This should deal with clipping gradients

In [226]:
# Create an optimizer with the desired parameters.
opt = tf.keras.optimizers.Adam(learning_rate=0.1, clipvalue=1)
var1, var2 = tf.Variable(1.0), tf.Variable(2.0)
# `loss` is a callable that takes no argument and returns the value
# to minimize.
loss = lambda: 3 * var1 * var1 + 2 * var2 * var2
# Call minimize to update the list of variables.
opt.minimize(loss, var_list=[var1, var2])
var1, var2

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.900001>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.900001>)